In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
import numpy as np
import pandas as pd

"The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

In [2]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')
low_sub_pf_isr_13078_nov15 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['propid'] == 13078) & (postflash_data['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['propid'] == 13078) & (postflash_data['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)
#print(low_sub_pf_isr_13078)

In [3]:
paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

In [4]:
low_sub_pf_isr_13078['count_rate'] = (low_sub_pf_isr_13078['sig_mean']/low_sub_pf_isr_13078['flash_dur']) * low_sub_pf_isr_13078['flash_dur']

In [5]:
low_sub_pf_isr_13078

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime,count_rate
rootname,,,,,,,,,,,,,,,,,,,,
ic6da6wzq,ic6da6wzq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:26:03,B,5,2.0,LOW,True,0.5,3.294864,3.049004,2.929958,2.929958,21.873594,2.442729,1.524502,2012-11-15 18:26:03,3.049004
ic6da3ufq,ic6da3ufq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:14:52,B,5,2.0,LOW,True,0.5,3.367939,3.133801,3.238117,3.238117,18.409138,2.436243,1.566901,2012-11-15 04:14:52,3.133801
ic6da3uhq,ic6da3uhq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:17:56,B,10,3.9,LOW,True,0.5,6.068842,5.892592,5.876036,5.876036,14.099956,2.814877,1.510921,2012-11-15 04:17:56,5.892592
ic6da3ujq,ic6da3ujq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:20:59,B,20,7.7,LOW,True,0.5,11.868539,11.529402,11.321764,11.321764,42.733799,3.393699,1.497325,2012-11-15 04:20:59,11.529402
ic6da6x1q,ic6da6x1q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:29:07,B,10,3.9,LOW,True,0.5,6.120877,5.883057,5.566433,5.566433,21.740099,2.798644,1.508476,2012-11-15 18:29:07,5.883057
ic6da3upq,ic6da3upq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:54:05,A,10,3.9,LOW,True,0.5,6.872035,6.259343,6.283906,6.283906,31.138172,2.834400,1.604960,2012-11-15 04:54:05,6.259343
ic6da6wqq,ic6da6wqq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:11:21,A,3,1.2,LOW,True,0.5,2.355810,2.006855,2.083049,2.083049,29.429537,2.304180,1.672379,2012-11-15 18:11:21,2.006855
ic6da3umq,ic6da3umq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,04:49:30,A,3,1.2,LOW,True,0.5,2.509913,1.991227,1.826905,1.826905,60.704193,2.307053,1.659356,2012-11-15 04:49:30,1.991227
ic6da6wyq,ic6da6wyq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2012-11-15,18:24:32,B,3,1.2,LOW,True,0.5,2.204074,1.892960,1.971943,1.971943,19.700354,2.295333,1.577467,2012-11-15 18:24:32,1.892960
